# A Novel Approach for Three-Way Classification of Lumbar Spine Degeneration Using Pseudo-Modality Learning to Handle Missing MRI Data

## Libs

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import KFold

## Training Pipeline

In [2]:
def train(embedding_path, labels_path,loss_fn = 'binary_crossentropy', n_splits=5):
    embeddings = pd.read_csv(embedding_path)
    labels = pd.read_csv(labels_path)
    
    id_cols = labels[['study_id', 'series_id']]
    cols_to_impute = labels.drop(columns=['study_id', 'series_id'])
    imputed_cols = cols_to_impute.apply(lambda x: x.fillna(x.mode()[0]))
    final_df = pd.concat([id_cols, imputed_cols], axis=1)
    labels = final_df

    id_cols = labels[['study_id', 'series_id']]
    cols_to_encode = labels.drop(columns=['study_id', 'series_id'])
    encoder = OneHotEncoder(sparse_output=False)
    encoded_cols = encoder.fit_transform(cols_to_encode)
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cols_to_encode.columns))
    final_df = pd.concat([id_cols, encoded_df], axis=1)
    df = pd.merge(embeddings, final_df, on='study_id', how='inner')

    fc_layer = len(df.columns[515:])
    X = df.iloc[:, :512].values
    Y = df.iloc[:, 515:].values

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_accuracies = []
    fold_val_accuracies = []
    
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = Sequential([
            Input(shape=(512,)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(fc_layer, activation='sigmoid')
        ])

        model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
        history = model.fit(X_train, Y_train, epochs=20, batch_size=16, verbose=0, validation_data=(X_test, Y_test))
        loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
        fold_accuracies.append(accuracy)
        val_accuracy = sum(history.history['val_accuracy'])/len(history.history['val_accuracy'])
        fold_val_accuracies.append(val_accuracy)
        print(f'Fold {fold}: Test Accuracy = {accuracy:.4f}, Validation Accuracy = {val_accuracy:.4f}')

    avg_accuracy = np.mean(fold_accuracies)
    avg_val_accuracy = np.mean(fold_val_accuracies)
    print(f'Average Test Accuracy across {n_splits} folds: {avg_accuracy:.4f}, Average Validation Accuracy across {n_splits} folds: {avg_val_accuracy:.4f}')
    return avg_accuracy, avg_val_accuracy


## Combination Network

In [3]:
attention_embeddings_paths = [
    '/kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_hist.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST1_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST1_attention_embeddings_hist.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST2_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST2_attention_embeddings_hist.csv'    
]

average_embeddings_paths = [
    '/kaggle/input/embeddings-for-rsna/at2-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/at2-hist/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st1-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st1-hist/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st2-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st2-hist/final_embeddings.csv'
]

medicalnet_embeddings_paths = [
    '/kaggle/input/medicalnet-attention-layers-for-rsna/AT2_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/AT2_attention_embeddings_hist.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST1_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST1_attention_embeddings_hist.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST2_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST2_attention_embeddings_hist.csv',
]

labels_paths = [
    '/kaggle/input/preprocessed-dataset/train_data_AT2.csv',
    '/kaggle/input/preprocessed-dataset/train_data_ST1.csv',
    '/kaggle/input/preprocessed-dataset/train_data_ST2.csv'
]

list_of_combination = [
    'AT2 - GSL - Attention Network',
    'AT2 - HIST - Attention Network',
    'ST1 - GSL - Attention Network',
    'ST1 - HIST - Attention Network',
    'ST2 - GSL - Attention Network',
    'ST2 - HIST - Attention Network',
    
    'AT2 - GSL - Average ResNet50',
    'AT2 - HIST - Average ResNet50',
    'ST1 - GSL - Average ResNet50',
    'ST1 - HIST - Average ResNet50',
    'ST2 - GSL - Average ResNet50',
    'ST2 - HIST - Average ResNet50',
    
    'AT2 - GSL - MedicalNet Network',
    'AT2 - HIST - MedicalNet Network',
    'ST1 - GSL - MedicalNet Network',
    'ST1 - HIST - MedicalNet Network',
    'ST2 - GSL - MedicalNet Network',
    'ST2 - HIST - MedicalNet Network'
]

## Results

In [4]:
results_df = pd.DataFrame(columns=['Combination', 'Avg_Test_Accuracy', 'Avg_Val_Accuracy'])

all_embedding_paths = [
    *attention_embeddings_paths,
    *average_embeddings_paths,
    *medicalnet_embeddings_paths
]

corresponding_labels_paths = [
    labels_paths[0],  # AT2 - GSL - Attention Network
    labels_paths[0],  # AT2 - HIST - Attention Network
    labels_paths[1],  # ST1 - GSL - Attention Network
    labels_paths[1],  # ST1 - HIST - Attention Network
    labels_paths[2],  # ST2 - GSL - Attention Network
    labels_paths[2],  # ST2 - HIST - Attention Network
    
    labels_paths[0],  # AT2 - GSL - Average ResNet50
    labels_paths[0],  # AT2 - HIST - Average ResNet50
    labels_paths[1],  # ST1 - GSL - Average ResNet50
    labels_paths[1],  # ST1 - HIST - Average ResNet50
    labels_paths[2],  # ST2 - GSL - Average ResNet50
    labels_paths[2],  # ST2 - HIST - Average ResNet50
    
    labels_paths[0],  # AT2 - GSL - MedicalNet Network
    labels_paths[0],  # AT2 - HIST - MedicalNet Network
    labels_paths[1],  # ST1 - GSL - MedicalNet Network
    labels_paths[1],  # ST1 - HIST - MedicalNet Network
    labels_paths[2],  # ST2 - GSL - MedicalNet Network
    labels_paths[2],  # ST2 - HIST - MedicalNet Network
]

for embedding_path, label_path, name in zip(all_embedding_paths, corresponding_labels_paths, list_of_combination):
    print(f"\nTraining for: {name} - {embedding_path}")
    avg_accuracy, avg_val_accuracy = train(embedding_path, label_path)
    
    result_row = pd.DataFrame({
        'Combination': [name],
        'Avg_Test_Accuracy': [avg_accuracy],
        'Avg_Val_Accuracy': [avg_val_accuracy]
    })
    
    results_df = pd.concat([results_df, result_row], ignore_index=True)

print("\nFinal Results DataFrame:")
results_df


Training for: AT2 - GSL - Attention Network - /kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_gsl.csv


I0000 00:00:1728147222.543772      69 service.cc:145] XLA service 0x7ef764003ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728147222.543816      69 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728147222.543820      69 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1728147225.168369      69 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Fold 1: Test Accuracy = 0.7052, Validation Accuracy = 0.5368
Fold 2: Test Accuracy = 0.3668, Validation Accuracy = 0.3759
Fold 3: Test Accuracy = 0.5712, Validation Accuracy = 0.3795
Fold 4: Test Accuracy = 0.4606, Validation Accuracy = 0.4013
Fold 5: Test Accuracy = 0.5034, Validation Accuracy = 0.3008
Average Test Accuracy across 5 folds: 0.5214, Average Validation Accuracy across 5 folds: 0.3988

Training for: AT2 - HIST - Attention Network - /kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_hist.csv


/tmp/ipykernel_23/118332904.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result_row], ignore_index=True)


Fold 1: Test Accuracy = 0.4054, Validation Accuracy = 0.2616
Fold 2: Test Accuracy = 0.5310, Validation Accuracy = 0.4523
Fold 3: Test Accuracy = 0.7990, Validation Accuracy = 0.5070
Fold 4: Test Accuracy = 0.5444, Validation Accuracy = 0.4964
Fold 5: Test Accuracy = 0.4597, Validation Accuracy = 0.3778
Average Test Accuracy across 5 folds: 0.5479, Average Validation Accuracy across 5 folds: 0.4190

Training for: ST1 - GSL - Attention Network - /kaggle/input/attention-embeddings-for-rsna/ST1_attention_embeddings_gsl.csv
Fold 1: Test Accuracy = 0.6332, Validation Accuracy = 0.5158
Fold 2: Test Accuracy = 0.5646, Validation Accuracy = 0.3693
Fold 3: Test Accuracy = 0.4433, Validation Accuracy = 0.3722
Fold 4: Test Accuracy = 0.5926, Validation Accuracy = 0.5507
Fold 5: Test Accuracy = 0.4180, Validation Accuracy = 0.3505
Average Test Accuracy across 5 folds: 0.5303, Average Validation Accuracy across 5 folds: 0.4317

Training for: ST1 - HIST - Attention Network - /kaggle/input/attention-

,Combination,Avg_Test_Accuracy,Avg_Val_Accuracy
0,AT2 - GSL - Attention Network,0.521442,0.398828
1,AT2 - HIST - Attention Network,0.547893,0.419021
2,ST1 - GSL - Attention Network,0.530349,0.431689
3,ST1 - HIST - Attention Network,0.501869,0.416668
4,ST2 - GSL - Attention Network,0.547462,0.486262
5,ST2 - HIST - Attention Network,0.523424,0.432608
6,AT2 - GSL - Average ResNet50,0.552959,0.576138
7,AT2 - HIST - Average ResNet50,0.450107,0.499419
8,ST1 - GSL - Average ResNet50,0.504431,0.507988
9,ST1 - HIST - Average ResNet50,0.435836,0.529633


## Random

In [ ]:
def random_predictions(path):
    labels = pd.read_csv(path)

    id_cols = labels[['study_id', 'series_id']]
    cols_to_impute = labels.drop(columns=['study_id', 'series_id'])
    imputed_cols = cols_to_impute.apply(lambda x: x.fillna(x.mode()[0]))
    final_df = pd.concat([id_cols, imputed_cols], axis=1)

    id_cols = final_df[['study_id', 'series_id']]
    cols_to_encode = final_df.drop(columns=['study_id', 'series_id'])
    encoder = OneHotEncoder(sparse_output=False)
    encoded_cols = encoder.fit_transform(cols_to_encode)
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cols_to_encode.columns))
    final_df = pd.concat([id_cols, encoded_df], axis=1)

    Y_true = final_df.drop(columns=['study_id', 'series_id']).values

    np.random.seed(42)  # For reproducibility
    random_predictions = np.random.rand(*Y_true.shape)  # Random floats in [0.0, 1.0]

    threshold = np.mean(random_predictions)

    binary_predictions = (random_predictions > threshold).astype(float)

    accuracy = np.mean(np.all(binary_predictions == Y_true, axis=1))

    print(f"Random Predictions Accuracy: {accuracy:.4f}")
    print(f"Self-adjusting Threshold: {threshold:.4f}")

In [ ]:
random_predictions('/kaggle/input/preprocessed-dataset/train_data_AT2.csv')
random_predictions('/kaggle/input/preprocessed-dataset/train_data_ST1.csv')
random_predictions('/kaggle/input/preprocessed-dataset/train_data_ST2.csv')

Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.4996
Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.4995
Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.5004
